In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


In [9]:

# 导入npy数据集
X_train = np.load('Assignment1-Dataset/train_data.npy')
X_test = np.load('Assignment1-Dataset/test_data.npy')
Y_train = np.load('Assignment1-Dataset/train_label.npy').flatten()
Y_test = np.load('Assignment1-Dataset/test_label.npy').flatten()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 假设你已经有这些 numpy 数据
# X_train, Y_train, X_test, Y_test
# 全部是 float32 的 X，int64 的 Y

# ===== 模型定义 =====
class SimpleNet(nn.Module):
    def __init__(self, input_dim=128, num_classes=10):
        super(SimpleNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# ===== 数据转换 =====
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=32)

# ===== 模型训练函数 =====
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        total_correct += (output.argmax(1) == y).sum().item()

    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

# ===== 测试函数 =====
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss = criterion(output, y)
            total_loss += loss.item() * x.size(0)
            total_correct += (output.argmax(1) == y).sum().item()

    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

# ===== 训练主程序 =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNet(input_dim=128, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}")


Epoch 01 | Train Loss: 1.7023, Acc: 0.3927 | Test Loss: 1.5232, Acc: 0.4562
Epoch 02 | Train Loss: 1.4948, Acc: 0.4650 | Test Loss: 1.4214, Acc: 0.4895
Epoch 03 | Train Loss: 1.4233, Acc: 0.4877 | Test Loss: 1.3882, Acc: 0.5042
Epoch 04 | Train Loss: 1.3866, Acc: 0.5041 | Test Loss: 1.3664, Acc: 0.5101
Epoch 05 | Train Loss: 1.3586, Acc: 0.5119 | Test Loss: 1.3537, Acc: 0.5143
Epoch 06 | Train Loss: 1.3349, Acc: 0.5222 | Test Loss: 1.3457, Acc: 0.5205
Epoch 07 | Train Loss: 1.3204, Acc: 0.5283 | Test Loss: 1.3435, Acc: 0.5207
Epoch 08 | Train Loss: 1.3062, Acc: 0.5325 | Test Loss: 1.3381, Acc: 0.5197
Epoch 09 | Train Loss: 1.2979, Acc: 0.5318 | Test Loss: 1.3315, Acc: 0.5225
Epoch 10 | Train Loss: 1.2875, Acc: 0.5390 | Test Loss: 1.3295, Acc: 0.5203
Epoch 11 | Train Loss: 1.2790, Acc: 0.5397 | Test Loss: 1.3307, Acc: 0.5262
Epoch 12 | Train Loss: 1.2683, Acc: 0.5428 | Test Loss: 1.3248, Acc: 0.5221
Epoch 13 | Train Loss: 1.2683, Acc: 0.5449 | Test Loss: 1.3196, Acc: 0.5330
Epoch 14 | T